## STAGE 4 : Portfolio Construction and Optimization

Purpose :

To transform predicted stock-level returns (alphas) into actual portfolio weights using both rule-based and optimization-based allocation schemes.
This stage bridges model predictions (Stage 3) with portfolio backtesting (Stage 5) by deciding how much to invest in each stock.

#### Overview of Stage 4

| Sub-Stage | Component              | Objective                               | Output                  |
| --------- | ---------------------- | --------------------------------------- | ----------------------- |
| **4.1**   | Portfolio Construction | Build heuristic (rule-based) portfolios | `weekly_weights.csv`    |
| **4.2**   | Portfolio Optimization | Build risk-return optimized portfolios  | `optimized_weights.csv` |


#### Concepts Invloved

* <b>Ranking and Allocation</b>: Selecting top-N assets by predicted alpha (expected return).

* <b>Weighting Schemes</b>:

    <em>Equal Weight (EW)</em> — risk-neutral, diversification-focused.

    <em>Predicted Return Weight (PRW)</em> — higher alpha → higher weight.

    <em>Volatility-Adjusted Weight (VAW)<em> — risk-aware weighting (lower vol → higher weight).

* <b>Mean-Variance Optimization (Markowitz, 1952)</b>:
Maximize return for a given level of risk using covariance matrix.

* <b>Transaction Costs & Turnover</b>: Adjust expected return for rebalancing costs.

* <b>Convex Optimization (via cvxpy)</b>: Efficient solver for constrained portfolio problems.

#### Input Files

| File                                | Description                                   | Source Stage |
| ----------------------------------- | --------------------------------------------- | ------------ |
| `artifacts/alpha_predictions.csv`   | Model-predicted expected returns              | Stage 3      |
| `results/volatility_matrix.csv`     | Stock-level volatility                        | Stage 3.5    |
| `results/factors/price_factors.csv` | Factor exposures (optional correlation proxy) | Stage 2      |


### Stage 4.1 - Portfolio Construction (Rule-Based)

#### Purpose :

To allocate portfolio weights using deterministic (non-optimized) methods for the top-N predicted stocks.

 Workflow Overview

| Step  | Task                 | Description                         | Output            |
| ----- | -------------------- | ----------------------------------- | ----------------- |
| 4.1.1 | Load Inputs          | Read alpha predictions & volatility | DataFrame         |
| 4.1.2 | Merge                | Combine on `date`, `ticker`         | Unified dataset   |
| 4.1.3 | Rank Stocks          | Sort by predicted return per week   | Ranked universe   |
| 4.1.4 | Select Top-N         | Keep top-N tickers each week        | Active universe   |
| 4.1.5 | Apply Weight Schemes | Compute EW, PRW, and VAW weights    | Weight columns    |
| 4.1.6 | Normalize            | Sum of weights = 1 per week         | Valid allocations |
| 4.1.7 | Export & Log         | Save weights and summary            | `.csv` outputs    |


### Stage 4.2 — Portfolio Optimization (Quantitative Approach)

#### Purpose:

Optimize portfolio weights mathematically using <b>Mean-Variance Optimization (MVO)</b> and <b> Minimum-Variance (MinVar) </b>approaches.

Workflow Overview

| Step  | Task                      | Description                            | Output               |
| ----- | ------------------------- | -------------------------------------- | -------------------- |
| 4.2.1 | Load Inputs               | Read alpha & volatility                | DataFrame            |
| 4.2.2 | Build Covariance Matrix   | Diagonal (vol²) or factor-based        | Risk matrix Σ        |
| 4.2.3 | Solve MVO                 | Maximize Sharpe (risk-adjusted return) | `w_mvo_best_sharpe`  |
| 4.2.4 | Solve MinVar              | Minimize portfolio variance            | `w_minvar`           |
| 4.2.5 | Apply Constraints         | Long-only, 10% max per stock           | Feasible allocations |
| 4.2.6 | Add Turnover Penalty      | Penalize large trades                  | Stable weights       |
| 4.2.7 | Compute Transaction Costs | Adjust expected return                 | Net Sharpe           |
| 4.2.8 | Export & Visualize        | Save weights + plots                   | `.csv` + `.png`      |
